In [1]:
from ucc_ft.surface_code import RotatedSurfaceCode
from ucc_ft.checker import ft_check_ideal_qasm

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


This notebook demonstrates how `ucc-ft` is used to verify the fault tolerance of a circuit. It focuses on the the rotated surface code for d=3

In [2]:
# d= 3 code, show the stabilizers for reference
sc = RotatedSurfaceCode(3)
sc.stabilizers()

[stim.PauliString("+ZZ"),
 stim.PauliString("+___X__X"),
 stim.PauliString("+_ZZ_ZZ"),
 stim.PauliString("+XX_XX"),
 stim.PauliString("+___ZZ_ZZ"),
 stim.PauliString("+____XX_XX"),
 stim.PauliString("+_______ZZ"),
 stim.PauliString("+__X__X")]

Let's start by checking the CNOT gate for this code, which is just implemented transversally.

First, let's load the QASM for that gate:

In [3]:
cnot_circuit = """
    OPENQASM 3.0;
    include "stdgates.inc";

    const uint d = 3;
    const uint data_size = d * d;
    qubit[data_size] state1;
    qubit[data_size] state2;

    def logical_CNOT() {
        // QASM ranges are inclusive for both start and end
        for int i in [0:(data_size-1)] {
            cx state1[i], state2[i];
        }
    }
    """

Now run the checker on that gate for this code:

In [4]:
ft_check_ideal_qasm(sc, cnot_circuit, "logical_CNOT", "gate")

Precompiling QuantumSE...
   1565.8 ms  ✓ QuantumSE
  1 dependency successfully precompiled in 2 seconds. 65 already precompiled.


shape: (18, 36)
shape: (18, 36)
>>> Pass!
>>> Pass!


[ Info: '`bitwuzla -rwl 1`' is used as smt solver for FT_condition case
[ Info: '`bitwuzla -rwl 1`' has solved the problem
[ Info: '`bitwuzla -rwl 1`' is used as smt solver for FT_condition case
[ Info: '`bitwuzla -rwl 1`' has solved the problem


True

A more complicated example is state preparation. For state preparation, we follow the Shor method, where the stabilizers are repeatedly measured, and then once the outcome stabilized, we apply a correciton get get back to the logical 0 state.

First, load the [QASM](../test/rotated_surface_code.qasm). Note that it has some `extern` functions for classical components. One is for rotating indicies into the stabilizer code. A more complex one is `mwpm_full` which represents the minimum weigh-perfect matching function used to diagnose the errors. For checking, we don't actually run the algorithm, but this function does act as an oracle and sets constraints on the classical symbolic variables to reflect the act of correcting.

In [5]:
qasm_prep = open("../test/rotated_surface_code.qasm").read()

To check the fault-tolerance belwo, there's a little more work involved, because these `extern` functions are defined in a separate Julia file. This also shows a bit more of the pipeline, where the QASM is converted into a `@qprog`, which is the embedded DSL in Julia the authors use to represent a quantum circuit. It is converted using 

In [6]:
from ucc_ft.checker import qasm_to_qprog_source, julia_source_to_qprog, ft_check_ideal

qprog_src = qasm_to_qprog_source(qasm_prep)

julia_source = open("../test/rotated_surface_code.in_translation.jl").read()

qprog_context = julia_source_to_qprog(
    julia_source + "\n\n" + qprog_src,
    [
        "prepare_state",
        "rotated_surface_z_m",
        "rotated_surface_x_m",
        "rotated_surface_lz_m",
        "prepare_cat",
        "rotate",
        "mwpm_full",
        "mwpm_full_x",
        "mwpm_full_z",
        "_xadj",
        "_zadj",
        "data_size",
        "cat_size",
        "state",
        "cat",
        "verify",
        "num_syndromes",
    ],
)

ft_check_ideal(
    sc,
    qprog_context.get_qprog("prepare_state"),
    qprog_context,
    "prepare",
    NERRS=12,
)

shape: (9, 18)
shape: (9, 18)
>>> Pass!


[ Info: '`bitwuzla -rwl 1`' is used as smt solver for FT_condition case
[ Info: '`bitwuzla -rwl 1`' has solved the problem


True

The conversion is done by first parsing the QASM to an Abstract Syntax Tree (AST) using `openqasm`. A custom visitor is then used to walk the AST and emit the corresponding qprog. This looks like:

In [7]:
print(qprog_src)

__qubit_count = 0
const d = 3;
const data_size = (d * d);
const cat_size = (d + 1);
const verify_size = 1;
const num_syndromes = ((((d * d) - 1)) ÷ 2);
state = [i + __qubit_count for i in 1:(data_size)]
__qubit_count += data_size;
cat = [i + __qubit_count for i in 1:(cat_size)]
__qubit_count += cat_size;
verify = __qubit_count + 1 
__qubit_count += 1;
@qprog prepare_cat (num_cat ) begin

  res = bv_val(ctx, 1, 1);
  @repeat begin 
    INIT(cat[( 0 ) + 1 ]);
    res = 0;
    H(cat[( 0 ) + 1 ]);
    for i in (1):((num_cat - 1)) 
      INIT(cat[( i ) + 1 ]);
      CNOT(cat[( 0 ) + 1 ], cat[( i ) + 1 ]);
    end
    for i in (1):((num_cat - 1)) 
      INIT(verify);
      CNOT(cat[( (i - 1) ) + 1 ], verify);
      CNOT(cat[( i ) + 1 ], verify);
      tmp = DestructiveM(verify);
      res = (res | tmp);
    end
  end :until (res == bv_val(ctx,0,1))
end
@qprog rotated_surface_z_m (idx ) begin

  num_cat = 2;
  if ((idx < (((d - 1)) ÷ 2))) 
    num_cat = 2;
    prepare_cat(num_cat);
    CZ(cat

### Appendix - QASM Challenges

This section is just to demo that the QASM code is valid QASM3 using the reference parser, but is currently unsupported by qiskit/pyqasm

In [8]:
import openqasm3

openqasm3.parse(qasm_prep)

Program(span=Span(start_line=2, start_column=0, end_line=284, end_column=0), statements=[Include(span=Span(start_line=3, start_column=0, end_line=3, end_column=22), annotations=[], filename='stdgates.inc'), ConstantDeclaration(span=Span(start_line=25, start_column=0, end_line=25, end_column=16), annotations=[], type=UintType(span=Span(start_line=25, start_column=6, end_line=25, end_column=6), size=None), identifier=Identifier(span=Span(start_line=25, start_column=826, end_line=25, end_column=826), name='d'), init_expression=IntegerLiteral(span=Span(start_line=25, start_column=15, end_line=25, end_column=15), value=3)), ConstantDeclaration(span=Span(start_line=26, start_column=0, end_line=26, end_column=28), annotations=[], type=UintType(span=Span(start_line=26, start_column=6, end_line=26, end_column=6), size=None), identifier=Identifier(span=Span(start_line=26, start_column=844, end_line=26, end_column=852), name='data_size'), init_expression=BinaryExpression(span=Span(start_line=26, 

In [9]:
import pyqasm

module = pyqasm.loads(qasm_prep)
module.validate()

ERROR:pyqasm: Error at line 50, column 0 in QASM file

 >>>>>> extern mwpm_full_x(uint, bit[num_syndromes]) -> bit[data_size];




ValidationError: Unsupported statement of type <class 'openqasm3.ast.ExternDeclaration'>

In [ ]:
from qiskit_qasm3_import import parse

circuit = parse(qasm_prep)

ConversionError: 25,0: node of type ConstantDeclaration is not supported